In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
plt.rc('text', usetex=True)
plt.rc('font', family='Serif')

import matplotlib as mpl 
mpl.rcParams['figure.figsize'] = [9, 9]
mpl.rcParams['font.size'] = 27

mpl.rcParams['savefig.dpi'] = 150 
mpl.rcParams['xtick.minor.visible'] = True
mpl.rcParams['ytick.minor.visible'] = True
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'

mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True

mpl.rcParams['xtick.major.size'] = 10 
mpl.rcParams['xtick.minor.size'] = 5 

mpl.rcParams['ytick.major.size'] = 10 
mpl.rcParams['ytick.minor.size'] = 5 

mpl.rcParams['xtick.labelsize'] = 23
mpl.rcParams['ytick.labelsize'] = 23

In [ ]:
GC_name = 'NGC3201'
data_path = os.getcwd()
fidanka = pd.read_csv('fiducial_lines.csv')

In [ ]:
df = pd.read_csv(GC_name + '_fitstars.dat')

In [ ]:
Coord = df[['x','y']].values
plt.scatter(Coord[:,0],Coord[:,1],s=0.1)
plt.xlabel("X coordinate")
plt.ylabel('Y coordinate');

In [ ]:
idx = 1500
plt.scatter(Coord[:,0],Coord[:,1],s=0.1)
plt.scatter(df.iloc[idx]['x'], df.iloc[idx]['y'],label='Target star')
plt.xlabel("X coordinate")
plt.ylabel('Y coordinate')
plt.legend();

In [ ]:
neighbors = df.iloc[np.argpartition(np.sum(np.square(df.iloc[idx][['x','y']].values - Coord),axis=1),50)[:50]]
plt.scatter(Coord[:,0],Coord[:,1],s=0.1,alpha=0.5)
plt.scatter(df.iloc[idx]['x'], df.iloc[idx]['y'],label='Target star')
plt.scatter(neighbors['x'], neighbors['y'],s=0.5,label='100 Neighbor stars')
plt.xlabel("X coordinate")
plt.ylabel('Y coordinate')
plt.legend();

In [ ]:
V_MSTO = 18.3
VI_MSTO = 0.70
V_SGB = 17.45
VI_SGB = 0.75
plt.plot(fidanka['vi'],fidanka['v'],c='r',label='fidanka',alpha=0.7)
plt.scatter(df['vi'],df['v'],s=0.5,alpha=0.7,c='grey')
plt.scatter(VI_MSTO+0.01,V_MSTO,s=20,c='g',label='MSTO')
plt.scatter(VI_SGB,V_SGB,s=20,c='b',label='SGB')
plt.legend()
plt.ylim(V_SGB+2,V_SGB-2)
plt.xlim(0.6,1)
plt.xlabel('F606W - F814W')
plt.ylabel('F606W');

In [ ]:
#determine theta
#Table From Bedin, L. R., et al 2005
from scipy.interpolate import interp1d
E_BV_list = np.array([0,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,1.0])
A_F606W_list = np.array([0.0,0.14,0.279,0.419,0.558,0.595,0.834,0.972,1.110,1.248,1.385,1.522,1.658,1.794,1.930,2.066,2.202,2.741])
A_F814W_list = np.array([0.0,0.092,0.184,0.276,0.367,0.459,0.550,0.641,0.732,0.823,0.913,1.004,1.094,1.184,1.274,1.364,1.454,1.811])
ff_606 = interp1d(E_BV_list, A_F606W_list)
ff_814 = interp1d(E_BV_list, A_F814W_list)
E_BV = 0.24
A_F814W = ff_814(E_BV)
A_F606W = ff_606(E_BV)
theta = np.arctan(A_F814W/(A_F606W - A_F814W))
theta

In [ ]:
R_mat = np.array([[np.cos(-theta), -np.sin(-theta)],[np.sin(-theta),np.cos(-theta)]])
Obs_Rot = np.matmul(R_mat,(df[['vi','v']].values - np.array([VI_SGB,V_SGB])).T)
fidanka_Rot = np.matmul(R_mat,(fidanka[['vi','v']].values - np.array([VI_SGB,V_SGB])).T)
df['xprime'] = Obs_Rot[0]
df['yprime'] = Obs_Rot[1]
neighbors = df.iloc[np.argpartition(np.sum(np.square(df.iloc[idx][['x','y']].values - Coord),axis=1),50)[:50]]

In [ ]:
plt.scatter(df['xprime'],df['yprime'],s=0.5,alpha=1,c='grey')
plt.plot(fidanka_Rot[0],fidanka_Rot[1],c='r',label='fidanka')
plt.scatter(df.iloc[idx]['xprime'], df.iloc[idx]['yprime'],label='Target star')
plt.scatter(neighbors['xprime'], neighbors['yprime'],s=5,label='50 Neighbor stars')
plt.xlim(-3.5,4)
plt.ylim(1,-1.5)
plt.xlabel('x\'')
plt.ylabel('y\'')
plt.legend();

In [ ]:
from scipy.interpolate import CubicSpline
ff = CubicSpline(fidanka_Rot[1],fidanka_Rot[0],extrapolate=True)
plt.scatter(df['xprime'] - ff(df['yprime']),df['yprime'],s=0.1,alpha=0.5)
plt.scatter(neighbors['xprime'] - ff(neighbors['yprime']), neighbors['yprime'],s=5,label='50 Neighbor stars')
plt.legend();

In [ ]:
plt.hist(neighbors['xprime'] - ff(neighbors['yprime']),bins=20)
plt.vlines(np.median(neighbors['xprime'] - ff(neighbors['yprime'])),0,30,'r')

In [ ]:
idxs = range(len(df))
delta_x = np.zeros(len(df))
start_time = time.time()
for idx in idxs:
    neighbors = df.iloc[np.argpartition(np.sum(np.square(df.iloc[idx][['x','y']].values - Coord),axis=1),50)[:50]]
    delta_x[idx] = np.median(neighbors['xprime'] - ff(neighbors['yprime']))
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
plt.scatter(df['xprime'] - delta_x, df['yprime'],s=0.5,alpha=1,c='grey')
plt.scatter(df['xprime'], df['yprime'],s=0.5,alpha=1,c='g')
plt.plot(fidanka_Rot[0],fidanka_Rot[1],c='r',label='fidanka')
plt.xlim(-3.5,3.5)
plt.ylim(2,-2)
plt.xlabel('x\'')
plt.ylabel('y\'')

In [ ]:
Obs_Rot_CR = df[['xprime','yprime']].values.T
Obs_Rot_CR[0] -= delta_x.T

In [ ]:
obs_CR = np.matmul(np.linalg.inv(R_mat),Obs_Rot_CR) + np.array([[VI_SGB,V_SGB]]).T

In [ ]:
V_MSTO = 18.3
VI_MSTO = 0.70
V_SGB = 17.45
VI_SGB = 0.75
plt.plot(fidanka['vi'],fidanka['v'],c='r',label='fidanka')
plt.scatter(obs_CR[0] ,obs_CR[1],s=0.5,alpha=0.5,c='g',label='DRCR')
plt.scatter(df['vi'],df['v'],s=0.5,alpha=0.5,c='b',label='Uncorrected')
plt.legend()
plt.ylim(V_SGB+3.5,V_SGB-3.5)
plt.xlim(0.6,1.2);

In [ ]:
data = {'v':obs_CR[1], 'vi':obs_CR[0], 'i': obs_CR[1] - obs_CR[0], 'x': df['x'].values, 'y':df['y'].values}
fitstars_DRCR = pd.DataFrame(data=data)
fitstars_DRCR.to_csv("{}_fitstars_DRCR.dat".format(GC_name),index=False)

In [ ]:
fitstars_DRCR_2dKS = fitstars_DRCR[(fitstars_DRCR['v'] < V_SGB + 2) & (fitstars_DRCR['v'] > V_SGB - 2)]
fitstars_DRCR_2dKS.to_csv("{}_fitstars_2dKS_DRCR.dat".format(GC_name),index=False)

In [ ]:
idx = 4000
plt.scatter(Coord[:,0],Coord[:,1],s=0.1)
plt.scatter(df.iloc[idx]['x'], df.iloc[idx]['y'],label='Target star')
plt.xlabel("X coordinate")
plt.ylabel('Y coordinate')
plt.legend();

In [ ]:
neighbors = df.iloc[np.argpartition(np.sum(np.square(df.iloc[idx][['x','y']].values - Coord),axis=1),50)[:50]]
plt.scatter(Coord[:,0],Coord[:,1],s=0.1,alpha=0.5)
plt.scatter(df.iloc[idx]['x'], df.iloc[idx]['y'],label='Target star')
plt.scatter(neighbors['x'], neighbors['y'],s=0.5,label='100 Neighbor stars')
plt.xlabel("X coordinate")
plt.ylabel('Y coordinate')
plt.legend();

In [ ]:
R_mat = np.array([[np.cos(-theta), -np.sin(-theta)],[np.sin(-theta),np.cos(-theta)]])
Obs_Rot = np.matmul(R_mat,(df[['vi','v']].values - np.array([VI_SGB,V_SGB])).T)
fidanka_Rot = np.matmul(R_mat,(fidanka[['vi','v']].values - np.array([VI_SGB,V_SGB])).T)
df['xprime'] = Obs_Rot[0]
df['yprime'] = Obs_Rot[1]
neighbors = df.iloc[np.argpartition(np.sum(np.square(df.iloc[idx][['x','y']].values - Coord),axis=1),50)[:50]]
plt.scatter(df['xprime'],df['yprime'],s=0.5,alpha=1,c='grey')
plt.plot(fidanka_Rot[0],fidanka_Rot[1],c='r',label='fidanka')
plt.scatter(df.iloc[idx]['xprime'], df.iloc[idx]['yprime'],label='Target star',c='b')
plt.scatter(neighbors['xprime'], neighbors['yprime'],s=5,label='50 Neighbor stars',c='orange')
plt.xlim(-3.5,4)
plt.ylim(1,-1.5)
plt.xlabel('x\'')
plt.ylabel('y\'')
plt.legend();

In [ ]:
from scipy.interpolate import CubicSpline
ff = CubicSpline(fidanka_Rot[1],fidanka_Rot[0],extrapolate=True)
plt.scatter(df.iloc[idx]['xprime'] - ff(df.iloc[idx]['yprime']), df.iloc[idx]['yprime'],label='Target star',c='b')
plt.scatter(df['xprime'] - ff(df['yprime']),df['yprime'],s=0.1,alpha=0.5,c='grey')
plt.scatter(neighbors['xprime'] - ff(neighbors['yprime']), neighbors['yprime'],s=5,label='50 Neighbor stars',c='orange')
plt.legend();

In [ ]:
plt.hist(neighbors['xprime'] - ff(neighbors['yprime']),bins=20)
plt.vlines(np.median(neighbors['xprime'] - ff(neighbors['yprime'])),0,30,'r')

In [ ]:
mpl.rcParams['figure.figsize'] = [18, 11]
mpl.rcParams['font.size'] = 20
plt.subplots_adjust(hspace=0.2, wspace=1)
ax1 = plt.subplot2grid(shape=(2,6), loc=(0,0), colspan=2)
ax2 = plt.subplot2grid((2,6), (0,2), colspan=2)
ax3 = plt.subplot2grid((2,6), (0,4), colspan=2)
ax4 = plt.subplot2grid((2,6), (1,1), colspan=2)
ax5 = plt.subplot2grid((2,6), (1,3), colspan=2)
ax1.plot(fidanka['vi'],fidanka['v'],c='r',label='fidanka',alpha=0.7)
ax1.scatter(df['vi'],df['v'],s=0.5,alpha=0.7,c='grey')
ax1.set_ylim(V_SGB+2,V_SGB-2)
ax1.set_xlim(0.6,1)
ax1.set_xlabel('F606W - F814W')
ax1.set_ylabel('F606W');
ax2.scatter(Coord[:,0],Coord[:,1],s=0.1,alpha=0.5,c='grey')
ax2.scatter(neighbors['x'], neighbors['y'],s=0.5,label='100 Neighbor stars',c='orange',alpha=0.7)
ax2.scatter(df.iloc[idx]['x'], df.iloc[idx]['y'],label='Target star',c='b',s=5)
ax2.set_xlabel("X coordinate")
ax2.set_ylabel('Y coordinate')
ax3.scatter(df['xprime'],df['yprime'],s=0.5,alpha=1,c='grey')
ax3.plot(fidanka_Rot[0],fidanka_Rot[1],c='r',label='fidanka',alpha=0.7)
ax3.scatter(neighbors['xprime'], neighbors['yprime'],s=3,label='50 Neighbor stars',c='orange',alpha=0.7)
ax3.scatter(df.iloc[idx]['xprime'], df.iloc[idx]['yprime'],label='Target star',s=10,c='b')
ax3.set_xlim(-3.5,4)
ax3.set_ylim(1,-1.5)
ax3.set_xlabel('x\'')
ax3.set_ylabel('y\'')
ax4.scatter(df['xprime'] - ff(df['yprime']),df['yprime'],s=0.1,alpha=0.5,c='grey')
ax4.scatter(neighbors['xprime'] - ff(neighbors['yprime']), neighbors['yprime'],s=5,label='50 Neighbor stars',c='orange',alpha=0.7)
ax4.scatter(df.iloc[idx]['xprime'] - ff(df.iloc[idx]['yprime']), df.iloc[idx]['yprime'],label='Target star',c='b')
ax4.set_xlabel(r'$\Delta$ x')
ax4.set_ylabel('y\'')
ax4.set_ylim(1,-1.5)
ax5.hist(neighbors['xprime'] - ff(neighbors['yprime']),bins=20,density=True)
ax5.vlines(np.median(neighbors['xprime'] - ff(neighbors['yprime'])),0,30,'r')
ax5.set_xlabel(r'$\Delta$ x')
ax5.set_ylabel('Counts')

In [ ]:
mpl.rcParams['figure.figsize'] = [15, 10]
mpl.rcParams['font.size'] = 20
fig, axs = plt.subplots(2, 3, layout="constrained")
axs[0,0].scatter(Coord[:,0],Coord[:,1],s=0.1,alpha=0.5,c='grey')
axs[0,0].scatter(neighbors['x'], neighbors['y'],s=0.5,label='100 Neighbor stars',c='orange',alpha=0.7)
axs[0,0].scatter(df.iloc[idx]['x'], df.iloc[idx]['y'],label='Target star',c='b',s=5)
axs[0,0].set_xlabel("X coordinate")
axs[0,0].set_ylabel('Y coordinate')
axs[0,1].scatter(df['xprime'],df['yprime'],s=0.5,alpha=1,c='grey')
axs[0,1].plot(fidanka_Rot[0],fidanka_Rot[1],c='r',label='fidanka',alpha=0.7)
axs[0,1].scatter(neighbors['xprime'], neighbors['yprime'],s=3,label='50 Neighbor stars',c='orange',alpha=0.7)
axs[0,1].scatter(df.iloc[idx]['xprime'], df.iloc[idx]['yprime'],label='Target star',s=10,c='b')
axs[0,1].set_xlim(-3.5,4)
axs[0,1].set_ylim(1,-1.5)
axs[0,1].set_xlabel('x\'')
axs[0,1].set_ylabel('y\'')
axs[1,0].scatter(df['xprime'] - ff(df['yprime']),df['yprime'],s=0.1,alpha=0.5,c='grey')
axs[1,0].scatter(neighbors['xprime'] - ff(neighbors['yprime']), neighbors['yprime'],s=5,label='50 Neighbor stars',c='orange',alpha=0.7)
axs[1,0].scatter(df.iloc[idx]['xprime'] - ff(df.iloc[idx]['yprime']), df.iloc[idx]['yprime'],label='Target star',c='b')
axs[1,0].set_xlabel(r'$\Delta$ x')
axs[1,0].set_ylabel('y\'')
axs[1,0].set_ylim(1,-1.5)
axs[1,1].hist(neighbors['xprime'] - ff(neighbors['yprime']),bins=20,density=True)
axs[1,1].vlines(np.median(neighbors['xprime'] - ff(neighbors['yprime'])),0,30,'r')
axs[1,1].set_xlabel(r'$\Delta$ x')

In [ ]:
DRCR_df = pd.read_csv(GC_name +'_fitstars_DRCR.dat')
DRCR_df

In [ ]:
fig, axs = plt.subplots(1, 2, layout="constrained",sharey=True)
axs[0].scatter(DRCR_df['vi'],DRCR_df['v'],s=0.1,label='Corrected')
axs[0].text(0.65,15.7,'Corrected',bbox=dict(pad=10, ec="k", fc="none"))
axs[1].scatter(df['vi'],df['v'],s=0.1,label='Uncorrected')
axs[1].text(0.65,15.7,'Uncorrected',bbox=dict(pad=10, ec="k", fc="none"))
fig.supxlabel('F606W - F814W')
fig.supylabel('F606W')
plt.ylim(17.45+2,17.45-2);

In [ ]:
plt.scatter(DRCR_df['vi'],DRCR_df['v'],s=0.1)